![A car dashboard with lots of new technical features.](images/dashboard.jpg)

A car manufacturer who is looking at implementing LLMs into vehicles to provide guidance to drivers. Integrating car manuals with OpenAI to create a context-aware chatbot. This context-aware LLM can be hooked up to a text-to-speech software to read the model's response aloud.

The manual, stored as an HTML file, `mg-zs-warning-messages.html`, is from an MG ZS, a compact SUV. Armed with your newfound knowledge of LLMs and LangChain, I am trying to implement Retrieval Augmented Generation (RAG) to create the context-aware chatbot.



In [3]:
# Update your environment to Python 3.10 as described above before running this cell
import subprocess
import pkg_resources

def install_if_needed(package, version):
    '''Function to ensure that the libraries used are consistent to avoid errors.'''
    try:
        pkg = pkg_resources.get_distribution(package)
        if pkg.version != version:
            raise pkg_resources.VersionConflict(pkg, version)
    except (pkg_resources.DistributionNotFound, pkg_resources.VersionConflict):
        try:
            subprocess.check_call(["pip", "install", f"{package}=={version}"], stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            print(f"Failed to install {package}=={version}. Error: {e}")

install_if_needed("langchain-core", "0.3.18")
install_if_needed("langchain-openai", "0.2.8")
install_if_needed("langchain-community", "0.3.7")
install_if_needed("unstructured", "0.14.4")
install_if_needed("langchain-chroma", "0.1.4")
install_if_needed("langchain-text-splitters", "0.3.2")

Defaulting to user installation because normal site-packages is not writeable
ERROR: Ignored the following yanked versions: 0.1.5, 0.2.16, 0.2.31
ERROR: Ignored the following versions that require a different python version: 0.3.0 Requires-Python <4.0,>=3.9; 0.3.0.dev1 Requires-Python <4.0,>=3.9; 0.3.0.dev2 Requires-Python <4.0,>=3.9; 0.3.0.dev3 Requires-Python <4.0,>=3.9; 0.3.0.dev4 Requires-Python <4.0,>=3.9; 0.3.0.dev5 Requires-Python <4.0,>=3.9; 0.3.1 Requires-Python <4.0,>=3.9; 0.3.10 Requires-Python <4.0,>=3.9; 0.3.11 Requires-Python <4.0,>=3.9; 0.3.12 Requires-Python <4.0,>=3.9; 0.3.13 Requires-Python <4.0,>=3.9; 0.3.14 Requires-Python <4.0,>=3.9; 0.3.15 Requires-Python <4.0,>=3.9; 0.3.16 Requires-Python <4.0,>=3.9; 0.3.17 Requires-Python <4.0,>=3.9; 0.3.18 Requires-Python <4.0,>=3.9; 0.3.19 Requires-Python <4.0,>=3.9; 0.3.2 Requires-Python <4.0,>=3.9; 0.3.20 Requires-Python <4.0,>=3.9; 0.3.21 Requires-Python <4.0,>=3.9; 0.3.22 Requires-Python <4.0,>=3.9; 0.3.23 Requires-Python 

In [ ]:
# Set your API key to a variable
import os
openai_api_key = os.environ["OPENAI_API_KEY"]

# Import the required packages
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

In [ ]:
# Load the HTML as a LangChain document loader
loader = UnstructuredHTMLLoader(file_path="data/mg-zs-warning-messages.html")
car_docs = loader.load()

In [ ]:
#Spliting the document
rc_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
)

splits = rc_splitter.split_documents(car_docs)
print(splits[0])

In [ ]:
# Initialize Chroma vectorstore with documents as splits and using OpenAIEmbeddings
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(openai_api_key=openai_api_key))

# Setup vectorstore as retriever
retriever = vectorstore.as_retriever()

In [ ]:
# Define RAG prompt
prompt = ChatPromptTemplate.from_template("You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:")


In [ ]:
# Initialize chat-based LLM with 0 temperature and using gpt-4o-mini
model = ChatOpenAI(openai_api_key=openai_api_key, model_name="gpt-4o-mini", temperature=0)

# Setup the chain
rag_chain = (
    {"context": retriever , "question": RunnablePassthrough()}
    | prompt
    | model
)

In [ ]:
# Initialize query
query = "The Gasoline Particular Filter Full warning has appeared. What does this mean and what should I do about it?"

# Invoke the query
answer = rag_chain.invoke(query).content
print(answer)